In [1]:
import sys
sys.path.append("../")
sys.path.append("../src")
sys.path.append("../data/embeddings")
sys.path.append("../data/embeddings/biasbios")
import classifier

import debias

import codecs
import json

import numpy as np
import random
import sklearn
from sklearn import model_selection
from sklearn import cluster
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from pytorch_transformers import BertTokenizer, BertModel, BertForMaskedLM

import scipy
from scipy import linalg
from scipy import sparse
from scipy.stats.stats import pearsonr
import tqdm
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier, SGDRegressor, Perceptron, LogisticRegression

%matplotlib inline
matplotlib.rcParams['agg.path.chunksize'] = 10000

import warnings
warnings.filterwarnings("ignore")

import pickle
from collections import defaultdict, Counter
from typing import List, Dict

import copy

import seaborn as sn
import pandas as pd

import eli5

In [2]:
PROFS = ['professor', 'physician', 'attorney', 'photographer', 'journalist', 'nurse', 'psychologist', 'teacher',
'dentist', 'surgeon', 'architect', 'painter', 'model', 'poet', 'filmmaker', 'software_engineer',
'accountant', 'composer', 'dietitian', 'comedian', 'chiropractor', 'pastor', 'paralegal', 'yoga_teacher',
'dj', 'interior_designer', 'personal_trainer', 'rapper']

PROF2UNIFIED_PROF = {"associate professor": "professor", "assistant professor": "professor", "software engineer": "software_engineer", "psychotherapist": "psychologist", "orthopedic surgeon": "surgeon", "trial lawyer": "attorney","plastic surgeon": "surgeon",  "trial attorney": "attorney", "senior software engineer": "software_engineer", "interior designer": "interior_designer", "certified public accountant": "accountant", "cpa": "accountant", "neurosurgeon": "surgeon", "yoga teacher": "yoga_teacher", "nutritionist": "dietitian", "personal trainer": "personal_trainer", "certified personal trainer": "personal_trainer", "yoga instructor": "yoga_teacher"}

In [3]:
def load_data(fname):
    with open(fname, "rb") as f:
        data = pickle.load(f)
    return data        
        
    
def count_profs_and_gender(data: List[dict]):
    
    counter = defaultdict(Counter)
    for entry in data:
        gender, prof = entry["gender"], entry["raw_title"]
        counter[prof.lower()][gender.lower()] += 1
        
    return counter

def filter_dataset(data, topk = 10):
    
    filtered = []
    counter = count_profs_and_gender(data)
    total_counts = [(prof, counter[prof]["f"] + counter[prof]["m"]) for prof in counter.keys()]
    profs_by_frq = sorted(total_counts, key = lambda x: -x[1])
    topk_profs = [p[0] for p in profs_by_frq[:topk]]
    
    print("Top-k professions: {}".format(topk_profs))
    for d in data:
        
        if d["raw_title"].lower() in topk_profs:
            filtered.append(d)
    
    return filtered

In [4]:
def load_dictionary(path):
    
    with open(path, "r", encoding = "utf-8") as f:
        
        lines = f.readlines()
        
    k2v, v2k = {}, {}
    for line in lines:
        
        k,v = line.strip().split("\t")
        v = int(v)
        k2v[k] = v
        v2k[v] = k
    
    return k2v, v2k

In [5]:
train = load_data('../data/biasbios/train.pickle')
dev = load_data('../data/biasbios/dev.pickle')
test = load_data('../data/biasbios/test.pickle')

In [6]:
p2i, i2p = load_dictionary("../data/biasbios/profession2index.txt")
g2i, i2g = load_dictionary("../data/biasbios/gender2index.txt")

In [7]:
train[0]

{'g': 'f',
 'p': 'professor',
 'text': 'Dr. Elizabeth Armstrong-Mensah is a clinical assistant professor in the Undergraduate Program at the School of Public Health at Georgia State University. She teaches Introduction to Chronic and Infectious Diseases, Health Equity and Disparities: Urban and Global Health Challenges, and Introduction to Program Implementation and Evaluation. Dr. Armstrong-Mensah previously taught Global Health to students enrolled in the Schools Master of Public Health Program.',
 'start': 153,
 'hard_text': 'She teaches Introduction to Chronic and Infectious Diseases, Health Equity and Disparities: Urban and Global Health Challenges, and Introduction to Program Implementation and Evaluation. Dr. Armstrong-Mensah previously taught Global Health to students enrolled in the Schools Master of Public Health Program.',
 'text_without_gender': '_ teaches Introduction to Chronic and Infectious Diseases, Health Equity and Disparities: Urban and Global Health Challenges, and

In [8]:
print(p2i)

{'accountant': 0, 'architect': 1, 'attorney': 2, 'chiropractor': 3, 'comedian': 4, 'composer': 5, 'dentist': 6, 'dietitian': 7, 'dj': 8, 'filmmaker': 9, 'interior_designer': 10, 'journalist': 11, 'model': 12, 'nurse': 13, 'painter': 14, 'paralegal': 15, 'pastor': 16, 'personal_trainer': 17, 'photographer': 18, 'physician': 19, 'poet': 20, 'professor': 21, 'psychologist': 22, 'rapper': 23, 'software_engineer': 24, 'surgeon': 25, 'teacher': 26, 'yoga_teacher': 27}


### get input representatons 

In [9]:
import nltk

def nltk_tokenization(text):
    tokens = nltk.word_tokenize(text)
    return tokens

def built_in_tokenization(text):
    tokens = text.split()
    return tokens

In [10]:
%%time
x_train = [x['hard_text_tokenized'] for x in train]
y_train = [x['p'] for x in train]

x_dev = [x['hard_text_tokenized'] for x in dev]
y_dev = [x['p'] for x in dev]

clf = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=built_in_tokenization)),
    ('selection', SelectKBest(chi2, k=10000)),
#     ('classifier', LogisticRegression())
    ('classifier', SGDClassifier(warm_start=True, loss='log', n_jobs=64, max_iter=75, random_state=0))
])

# clf.fit(x_train[:10000], y_train[:10000])
clf.fit(x_train, y_train)
clf.score(x_dev, y_dev)

CPU times: user 6min 11s, sys: 5.86 s, total: 6min 17s
Wall time: 29.6 s


0.8065138966515929

In [11]:
x_test = [x['hard_text'] for x in test]
y_test = [x['p'] for x in test]

In [12]:
clf.score(x_test, y_test)

0.7828499984752844

### perform debiasing

### Old  Debias

In [13]:
from src import old_debias

In [14]:
x_train_one_hot = clf.named_steps['selection'].transform(clf.named_steps['vectorizer'].transform([x['hard_text_tokenized'] for x in train]))
x_dev_one_hot = clf.named_steps['selection'].transform(clf.named_steps['vectorizer'].transform([x['hard_text_tokenized'] for x in dev]))

In [15]:
def get_projection_matrix(num_clfs, X_train, Y_train, X_dev, Y_dev, Y_train_main, Y_dev_main, dim=300):

    is_autoregressive = True
    reg = "l2"
    min_acc = 0.
    noise = False
    random_subset = False
    regression = False
    
    clf = SGDClassifier
    params = {'warm_start': True, 'loss': 'log', 'n_jobs': 64, 'max_iter': 100, 'random_state': 0}

    P = old_debias.get_debiasing_projection(clf, params, num_clfs, dim, is_autoregressive,
                                           min_acc, X_train, Y_train, X_dev, Y_dev,
                                           by_class=True, Y_train_main=Y_train_main, Y_dev_main=Y_dev_main)
    return P



num_clfs = 40
Y_dev_gender = np.array([d["g"] for d in dev])
Y_train_gender = np.array([d["g"] for d in train])
Y_dev_prof = np.array([d["p"] for d in dev])
Y_train_prof = np.array([d["p"] for d in train])

n_examples = 50000

In [ ]:
# To calculate a new projection matrix
# %%time

P = get_projection_matrix(40, x_train_one_hot[:n_examples],
                          Y_train_gender[:n_examples], x_dev_one_hot[:n_examples], Y_dev_gender[:n_examples],
                             Y_train_prof[:n_examples], Y_dev_prof[:n_examples], dim = 10000)

iteration: 0, accuracy: 0.9969259692088817:   2%|▎         | 1/40 [00:36<23:39, 36.39s/it]

done


iteration: 1, accuracy: 0.9922768152024796:   5%|▌         | 2/40 [03:38<50:40, 80.02s/it]

done


iteration: 2, accuracy: 0.9701488745490575:   8%|▊         | 3/40 [06:38<1:07:48, 109.96s/it]

...

iteration: 34, accuracy: 0.5872923123825009:  88%|████████▊ | 35/40 [1:47:01<16:20, 196.16s/it]

done


iteration: 35, accuracy: 0.5820588384736548:  90%|█████████ | 36/40 [1:50:14<13:00, 195.12s/it]

done


iteration: 36, accuracy: 0.5787815659773385:  90%|█████████ | 36/40 [1:52:40<13:00, 195.12s/it]

In [ ]:
# To load the already calculated matrix
P = np.load('P_linear.npy')

In [ ]:
P

In [ ]:
P.sum()

### Fine tuning before and after projection

In [ ]:
debiased_train = x_train_one_hot.dot(P)
debiased_dev = x_dev_one_hot.dot(P)

In [ ]:
params = {'warm_start': True, 'loss': 'log', 'n_jobs': 64, 'max_iter': 75, 'random_state': 0}
temp = SGDClassifier(**params)

temp.fit(x_train_one_hot[:n_examples], Y_train_gender[:n_examples])
temp.score(x_dev_one_hot, Y_dev_gender)

In [ ]:
params = {'warm_start': True, 'loss': 'log', 'n_jobs': 64, 'max_iter': 75, 'random_state': 0}
temp = SGDClassifier(**params)

temp.fit(debiased_train[:n_examples], Y_train_gender[:n_examples])
temp.score(debiased_dev, Y_dev_gender)

In [ ]:
params = {'warm_start': True, 'loss': 'log', 'n_jobs': 64, 'max_iter': 75, 'random_state': 0}
temp = SGDClassifier(**params)

temp.fit(x_train_one_hot[:n_examples], Y_train_prof[:n_examples])
temp.score(x_dev_one_hot, Y_dev_prof)

In [ ]:
params = {'warm_start': True, 'loss': 'log', 'n_jobs': 64, 'max_iter': 75, 'random_state': 0}
temp = SGDClassifier(**params)

temp.fit(debiased_train[:n_examples], Y_train_prof[:n_examples])
temp.score(debiased_dev, Y_dev_prof)

### New model - with debisaing

In [ ]:
svc = clf.named_steps['classifier']

In [ ]:
from copy import deepcopy

debiased_svc = deepcopy(svc)

In [ ]:
debiased_svc.coef_ = svc.coef_.dot(P.T)

In [ ]:
debias_clf = Pipeline([
    ('vectorizer', clf.named_steps['vectorizer']),
    ('selection', clf.named_steps['selection']),
    ('classifier', debiased_svc),
])

In [ ]:
clf.score(x_test, y_test)

In [ ]:
debias_clf.score(x_test, y_test)

In [ ]:
debias_ft_clf = Pipeline([
    ('vectorizer', clf.named_steps['vectorizer']),
    ('selection', clf.named_steps['selection']),
    ('classifier', debiased_svc),
])

#### test model without finetuning

In [ ]:
acc_test_bias = clf.score(x_test, y_test)
acc_test_debias = debias_clf.score(x_test, y_test)

print(acc_test_bias, acc_test_debias, acc_test_bias - acc_test_debias)

In [ ]:
reg_cls = clf.named_steps['classifier']
debias_cls = debias_clf.named_steps['classifier']

In [ ]:
vectorizer = clf.named_steps['vectorizer']
selector = clf.named_steps['selection']
feat_names = np.array(vectorizer.get_feature_names())

In [ ]:
with open('vocab_10k.txt', 'w') as f:
    for w in feature_names:
        f.write(w + '\n')
    

### Maximum weight change

In [ ]:
np.argsort(np.abs(debias_cls.coef_ - reg_cls.coef_)[0, :])[::-1]

In [ ]:
# a specific profession most changed words
for ind in np.argsort(np.abs(debias_cls.coef_ - reg_cls.coef_)[2, :])[::-1][:20]:
    print(feat_names[selector.get_support()][ind], ind)

In [ ]:
feature_names = feat_names[selector.get_support()]
coefs_diff = np.abs(debias_cls.coef_ - reg_cls.coef_)

In [ ]:
top_debiased_words = defaultdict(int)
for i in range(reg_cls.coef_.shape[0]):
    for ind_val, index in enumerate(np.argsort(coefs_diff[i, :])[::-1]):
        top_debiased_words[feature_names[index]] += coefs_diff[i, index]
        
for k, v in top_debiased_words.items():
    top_debiased_words[k] /= float(reg_cls.coef_.shape[0])

In [ ]:
import operator
sorted_x = sorted(top_debiased_words.items(), key=operator.itemgetter(1), reverse=True)
print([x[0] for x in sorted_x[:100]])

In [ ]:
for ind, (x, _) in enumerate(sorted_x[:100]):
    print(x + ', ', end='')
    if ind > 0 and ind % 6 == 0:
        print('\\\\')

In [ ]:
def read_gender_file(in_f):
    with open(in_f) as f:
        data = f.readlines()
    words = [x.strip() for x in data]
    return words[:500]
        
    
female_words = read_gender_file('female-biased.glove.1000.txt')
male_words = read_gender_file('male-biased.glove.1000.txt')

In [ ]:
len(male_words)

In [ ]:
len(female_words)

In [ ]:
male_words[:10]

In [ ]:
female_words[:10]

In [ ]:
with open('self_vocab.txt', 'w') as f:
    for x in feat_names[selector.get_support()]:
        f.write(x + '\n')

In [ ]:
gendered_words = male_words + female_words

feats_name_10k = feat_names[selector.get_support()]
gendered_feats = []
for gen in gendered_words:
    val = np.where(feats_name_10k == gen)[0]
    if len(val) > 0:
        gendered_feats.append(val[0])

In [ ]:
np.random.seed(1)
no_gendered_feats = np.random.randint(10000, size=500)

In [ ]:
len(gendered_feats)

In [ ]:
len(no_gendered_feats)

In [ ]:
gender_change = 0
gender_change_list = np.abs(debias_cls.coef_ - reg_cls.coef_)[5, :]
for ind in gendered_feats:
    gender_change += gender_change_list[ind]
    
gender_change /= len(gendered_feats)
print(gender_change)

In [ ]:
gender_change_list.mean()

In [ ]:
gender_change / gender_change_list.mean()

In [ ]:
profession_change = []
for i in range(reg_cls.coef_.shape[0]):
    gender_change_list = np.abs(debias_cls.coef_ - reg_cls.coef_)[i, :]
    gender_change = 0
    for ind in gendered_feats:
        gender_change += gender_change_list[ind]
    gender_change /= len(gendered_feats)
    profession_change.append(gender_change)

In [ ]:
profession_no_change = []
for i in range(reg_cls.coef_.shape[0]):
    gender_unchange_list = np.abs(debias_cls.coef_ - reg_cls.coef_)[i, :]
    gender_unchange = 0
    for ind in no_gendered_feats:
        gender_unchange += gender_unchange_list[ind]
    gender_unchange /= len(no_gendered_feats)
    profession_no_change.append(gender_unchange)

In [ ]:
mean_word_change = np.abs(debias_cls.coef_ - reg_cls.coef_).mean(axis=1)

profession_change_diff = np.array(profession_change) / mean_word_change
profession_unchange_diff = np.array(profession_no_change) / mean_word_change

In [ ]:
import seaborn as sns
# sns.set(style="darkgrid")

In [ ]:
def data2plot(change, unchange, professions):
    

#     ax = plt.plot(professions, [1] * len(change), '.--', label='mean change')
    ax = plt.plot(professions, change, '*:', label='biased words', linestyle='None')
    ax = plt.plot(professions, unchange, '+:', label='random words', linestyle='None')

#     plt.annotate('100%', size=8,
#     ha = 'center', va = 'bottom',
#     xytext = (65, 0.455),
#     xy = (75.2, 0.479),
#     arrowprops = { 'facecolor' : 'black', 'shrink' : 0.001, 'width': 0.5, 'headwidth': 6, 'headlength': 8 })

    plt.ylabel('Change relative to mean')
    plt.xlabel('Profession')
    plt.title('Relative weight change')
    
    plt.axhline(y=1, xmax=1.0, color='black', ls='--')
    plt.text(25, 1.025, 'no-change', fontsize=10, va='center', ha='center')#, backgroundcolor='')
    
#     plt.ylim(0.5,0.90)
    plt.legend()
    plt.xticks(rotation=75, fontsize=9)
    plt.ylim(0.8, 1.55)
    
    plt.savefig('diff_change.pdf', bbox_inches='tight')
    plt.show()
    
data2plot(profession_change_diff, profession_unchange_diff, list(p2i.keys()))

In [ ]:
profession_change_diff.mean()

In [ ]:
profession_unchange_diff.mean()

### TPR Calc

In [ ]:
def count_profs_and_gender(data: List[dict]):
    
    counter = defaultdict(Counter)
    for entry in data:
        gender, prof = entry["g"], entry["p"]
        counter[prof][gender] += 1
        
    return counter

prof_gender_count = count_profs_and_gender(train+dev+test)

In [ ]:
i2p

In [ ]:
prof_gender_count

In [ ]:
f,m = 0., 0.
prof2fem = dict()

for k, values in prof_gender_count.items():
    f += values['f']
    m += values['m']
    prof2fem[k] = values['f']/(values['f'] + values['m'])

print(f / (f + m))
print(prof2fem)

In [ ]:
def get_TPR(y_pred, y_true, p2i, i2p, gender):
    
    scores = defaultdict(Counter)
    prof_count_total = defaultdict(Counter)
    
    for y_hat, y, g in zip(y_pred, y_true, gender):
        
        if y == y_hat:
            
            scores[p2i[y]][g] += 1
        
        prof_count_total[p2i[y]][g] += 1
    
    tprs = defaultdict(dict)
    tprs_change = dict()
    tprs_ratio = []
    
    for profession, scores_dict in scores.items():
        
        good_m, good_f = scores_dict["m"], scores_dict["f"]
        prof_total_f = prof_count_total[profession]["f"]
        prof_total_m = prof_count_total[profession]["m"]
        tpr_m = (good_m) / prof_total_m
        tpr_f = (good_f) / prof_total_f
        
        tprs[profession]["m"] = tpr_m
        tprs[profession]["f"] = tpr_f
#         print(tpr_m, tpr_f)
        tprs_ratio.append(tpr_m/tpr_f)
#         tprs_ratio.append(tpr_f/tpr_m)
        tprs_change[profession] = tpr_m - tpr_f
        
    return tprs, tprs_change, np.mean(np.abs(tprs_ratio))


In [ ]:
k = len(dev)
y_pred = debias_clf.predict(x_dev[:k])
tprs_debias, tprs_change_debias, mean_tprs_debias = get_TPR(y_pred, y_dev[:k], p2i, i2p, [x['g'] for x in dev[:k]])

In [ ]:
k = len(dev)
y_pred = clf.predict(x_dev[:k])
tprs_biased, tprs_change_biased, mean_tprs_biased = get_TPR(y_pred, y_dev[:k], p2i, i2p, [x['g'] for x in dev[:k]])

In [ ]:
k = len(test)
y_pred = clf.predict(x_test[:k])
tprs_biased, tprs_change_biased, mean_tprs_biased = get_TPR(y_pred, y_test[:k], p2i, i2p, [x['g'] for x in test[:k]])

In [ ]:
k = len(test)
y_pred = debias_clf.predict(x_test[:k])
tprs_debias, tprs_change_debias, mean_tprs_debias = get_TPR(y_pred, y_test[:k], p2i, i2p, [x['g'] for x in test[:k]])

In [ ]:
def similarity_vs_tpr(tprs, title, measure, prof2fem):
    
    professions = list(tprs.keys())
    #
    """ 
    sims = dict()
    gender_direction = word2vec["he"] - word2vec["she"]
    
    for p in professions:
        sim = word2vec.cosine_similarities(word2vec[p], [gender_direction])[0]
        sims[p] = sim
    """
    tpr_lst = [tprs[p] for p in professions]
    sim_lst = [prof2fem[i2p[p]] for p in professions]
    
#     print(sim_lst)
    print(tpr_lst)

    #professions = [p.replace("_", " ") for p in professions if p in word2vec]
    
    plt.plot(sim_lst, tpr_lst, marker = "o", linestyle = "none")
    plt.xlabel("% women", fontsize = 13)
    plt.ylabel("{}_diff_female {}".format(measure, title), fontsize = 13)
#     for p in professions:
#         x,y = prof2fem[p], tprs[p]
#         plt.annotate(p , (x,y), size = 4, color = "red")
#     plt.savefig("{}_vs_bias_{}".format(measure, title), dpi = 600)
    print("Correlation: {}; p-value: {}".format(*pearsonr(sim_lst, tpr_lst)))
    plt.show()

In [ ]:
similarity_vs_tpr(tprs_change_biased, '', 'a', prof2fem)

In [ ]:
similarity_vs_tpr(tprs_change_debias, '', 'a', prof2fem)

### RMS

In [ ]:
def rms_diff(tpr_diff):
    return np.sqrt(np.mean(tpr_diff**2))

In [ ]:
print('before', rms_diff(np.array(list(tprs_change_biased.values()))))
print('after', rms_diff(np.array(list(tprs_change_debias.values()))))

### Perform finetuning

In [ ]:
clf.fit(X_train.dot(P), Y_train)

#### test again

In [ ]:
print(clf.score(X_dev.dot(P), Y_dev))